In [3]:
#imports
import os
import psycopg2
import pandas as pd
import numpy as np
from dotenv import load_dotenv
from sqlalchemy import create_engine
from sklearn.preprocessing import OneHotEncoder


load_dotenv()


True

In [2]:
def get_csv(region='NA1',game_mode='ARAM',patch='14.13'):
    #gets data collected into a csv

    #if using sqlalchemy
    # engine_name = f"postgresql://{os.getenv('DB_USER')}:{os.getenv('DB_PASSWORD')}@{os.getenv('DB_HOST')}:{os.getenv('DB_PORT')}/{os.getenv('DB_NAME')}"
    conn = psycopg2.connect(
        database = os.getenv('DB_NAME'),
        host = os.getenv('DB_HOST'),
        user = os.getenv('DB_USER'),
        password = os.getenv('DB_PASSWORD'),
        port = os.getenv('5432')
    )

    os.makedirs('MatchData', exist_ok=True)
    csv_path = f'MatchData/{region}_{game_mode}_{patch}.csv'

    cursor = conn.cursor()
    patch = patch+'%'

    query_sql = """SELECT * 
    FROM match_data 
    WHERE region = %s 
    AND game_mode = %s 
    AND patch LIKE %s"""

    query = cursor.mogrify(query_sql,(region, game_mode,patch))
    query = query.decode('utf-8')

    try:
        if os.path.exists(csv_path):
            print("Csv found")
        else:
            with open(csv_path,'w') as f:
                cursor.copy_expert("COPY ({}) TO STDOUT WITH CSV HEADER".format(query),f)
            print("Copy to csv successful")

    except (Exception, psycopg2.DatabaseError) as error:
        print(error)


In [4]:
def get_data(region='NA1',game_mode='ARAM',elo='ANY',version='14.13'):
    #features extraction, encoding and data verification
    engine_name = f"postgresql://{os.getenv('DB_USER')}:{os.getenv('DB_PASSWORD')}@{os.getenv('DB_HOST')}:{os.getenv('DB_PORT')}/{os.getenv('DB_NAME')}"
    engine = create_engine(engine_name)
    
    version = version+'%'
    elo = elo+'%'

    if(elo == 'ANY%'):
        query_sql = """SELECT * 
        FROM match_data 
        WHERE region = %s 
        AND game_mode = %s 
        AND version LIKE %s"""
        params = (region,game_mode,version)
        
    else:
        query_sql = """SELECT * 
        FROM match_data 
        WHERE region = %s 
        AND game_mode = %s 
        AND elo LIKE %s
        AND version LIKE %s"""
        params = (region, game_mode, elo, version)

    df = pd.read_sql_query(query_sql,con=engine,params=params)
    return df

In [109]:
def verify_data(df = None):
    df = df.dropna()

    all_champions = np.array(['Aatrox', 'Ahri', 'Akali', 'Akshan', 'Alistar', 'Amumu', 'Anivia',
                    'Annie', 'Aphelios', 'Ashe', 'AurelionSol', 'Azir', 'Bard',
                    'Belveth', 'Blitzcrank', 'Brand', 'Braum', 'Briar', 'Caitlyn',
                    'Camille', 'Cassiopeia', 'Chogath', 'Corki', 'Darius', 'Diana',
                    'DrMundo', 'Draven', 'Ekko', 'Elise', 'Evelynn', 'Ezreal',
                    'FiddleSticks', 'Fiora', 'Fizz', 'Galio', 'Gangplank', 'Garen',
                    'Gnar', 'Gragas', 'Graves', 'Gwen', 'Hecarim', 'Heimerdinger',
                    'Hwei', 'Illaoi', 'Irelia', 'Ivern', 'Janna', 'JarvanIV', 'Jax',
                    'Jayce', 'Jhin', 'Jinx', 'KSante', 'Kaisa', 'Kalista', 'Karma',
                    'Karthus', 'Kassadin', 'Katarina', 'Kayle', 'Kayn', 'Kennen',
                    'Khazix', 'Kindred', 'Kled', 'KogMaw', 'Leblanc', 'LeeSin',
                    'Leona', 'Lillia', 'Lissandra', 'Lucian', 'Lulu', 'Lux',
                    'Malphite', 'Malzahar', 'Maokai', 'MasterYi', 'Milio',
                    'MissFortune', 'MonkeyKing', 'Mordekaiser', 'Morgana', 'Naafiri',
                    'Nami', 'Nasus', 'Nautilus', 'Neeko', 'Nidalee', 'Nilah',
                    'Nocturne', 'Nunu', 'Olaf', 'Orianna', 'Ornn', 'Pantheon', 'Poppy',
                    'Pyke', 'Qiyana', 'Quinn', 'Rakan', 'Rammus', 'RekSai', 'Rell',
                    'Renata', 'Renekton', 'Rengar', 'Riven', 'Rumble', 'Ryze',
                    'Samira', 'Sejuani', 'Senna', 'Seraphine', 'Sett', 'Shaco', 'Shen',
                    'Shyvana', 'Singed', 'Sion', 'Sivir', 'Skarner', 'Smolder', 'Sona',
                    'Soraka', 'Swain', 'Sylas', 'Syndra', 'TahmKench', 'Taliyah',
                    'Talon', 'Taric', 'Teemo', 'Thresh', 'Tristana', 'Trundle',
                    'Tryndamere', 'TwistedFate', 'Twitch', 'Udyr', 'Urgot', 'Varus',
                    'Vayne', 'Veigar', 'Velkoz', 'Vex', 'Vi', 'Viego', 'Viktor',
                    'Vladimir', 'Volibear', 'Warwick', 'Xayah', 'Xerath', 'XinZhao',
                    'Yasuo', 'Yone', 'Yorick', 'Yuumi', 'Zac', 'Zed', 'Zeri', 'Ziggs',
                    'Zilean', 'Zoe', 'Zyra'])
    
    blue_team = ['blue_one','blue_two','blue_three','blue_four', 'blue_five']
    red_team = ['red_one', 'red_two', 'red_three', 'red_four', 'red_five']

    blue_team_encoded = np.zeros((len(df),len(all_champions)))
    blue_team_columns = [f"blue_{champ}" for champ in all_champions]
    red_team_encoded = np.zeros((len(df),len(all_champions)))
    red_team_columns = [f"red_{champ}" for champ in all_champions]


    for idx,row in df.iterrows():
        for col in blue_team:
            champ = row[col]
            champ_index = np.where(all_champions == champ)[0]
            blue_team_encoded[idx][champ_index] = 1

        for col in red_team:
            champ = row[col]
            champ_index = np.where(all_champions == champ)[0]
            red_team_encoded[idx][champ_index] = 1

    blue_team_encoded = pd.DataFrame(blue_team_encoded,columns=blue_team_columns)
    red_team_encoded = pd.DataFrame(red_team_encoded, columns=red_team_columns)

    df = df.drop(columns=['id','match_id','region','game_mode','elo','version'])
    df = df.drop(columns=blue_team)
    df = df.drop(columns=red_team)
    df = pd.concat([df,blue_team_encoded,red_team_encoded],axis=1)

    return df

In [110]:
def main(region,game_mode,elo,version):
    #get_csv(region=region,game_mode=game_mode,patch=patch)
    #data extraction
    df = get_data(region,game_mode,elo,version)
    #data encoding + validation
    df = verify_data(df)
    #build model + training 
    #model validation
    print(df.shape)
    print(df.head())
main('NA1','ARAM','ANY','14.13')

(5072, 345)
    blue_one     blue_two blue_three    blue_four blue_five red_one  \
0  Seraphine  MissFortune       Ekko  Mordekaiser     Galio  Graves   
1   Katarina         Nami       Nunu     Tristana     Galio   Akali   
2       Zyra        Yasuo    Morgana         Azir     Brand    Sion   
3      Karma         Sona       Fizz      Nidalee       Jax  Ezreal   
4      Corki     Vladimir    Caitlyn         Hwei     Amumu  Samira   

     red_two  red_three red_four red_five  ...  red_Yone  red_Yorick  \
0     Xerath     Veigar    Milio    Varus  ...       0.0         0.0   
1    Morgana       Rell   Ezreal  Skarner  ...       0.0         0.0   
2  TahmKench      Sivir   Kennen    Senna  ...       0.0         0.0   
3      Quinn       Sett    Shaco  Morgana  ...       0.0         0.0   
4      Galio  Gangplank    Xayah  Orianna  ...       0.0         0.0   

   red_Yuumi  red_Zac  red_Zed  red_Zeri  red_Ziggs  red_Zilean  red_Zoe  \
0        0.0      0.0      0.0       0.0        0.0 